In [1]:
from nustar_pysolar import planning, io
from imp import reload
reload(planning)
import astropy.units as u

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_197.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [4]:
tstart = '2017-07-18T12:00:00'
tend = '2017-07-18T20:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [5]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 5.219171016059789 deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [6]:
offset = [-190., -47.]*u.arcsec

# Loop over each orbit and see what the difference between the two methods is

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

*Looks like a fixed shift...probably some time-ephemeris issue.*

In [7]:
from astropy.coordinates import SkyCoord
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    skyfield_pos = planning.get_skyfield_position(midTime, offset, load_path='../data')
    print('SkyField Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), skyfield_pos[0], skyfield_pos[1]))
    skyfield_ephem = SkyCoord(skyfield_pos[0], skyfield_pos[1])
    sunpy_ephem = SkyCoord(sky_pos[0], sky_pos[1])
    print("")
    print("Offset between SkyField and Astropy: {} arcsec".format(skyfield_ephem.separation(sunpy_ephem).arcsec))
    print("")
         

Orbit: 0
Orbit start: 2017-07-18T11:27:00 Orbit end: 2017-07-18T12:28:50
Aim time: 2017-07-18T11:57:55 RA (deg): 117.84681928915637 deg Dec (deg): 20.96388011768501 deg
SkyField Aim time: 2017-07-18T11:57:55 RA (deg): 117.85271277014857 deg Dec (deg): 20.962841263623886 deg

Offset between SkyField and Astropy: 20.162088182148317 arcsec

Orbit: 1
Orbit start: 2017-07-18T13:03:40 Orbit end: 2017-07-18T14:05:30
Aim time: 2017-07-18T13:34:35 RA (deg): 117.91422476578671 deg Dec (deg): 20.951959754615196 deg
SkyField Aim time: 2017-07-18T13:34:35 RA (deg): 117.92011737743438 deg Dec (deg): 20.950918573733055 deg

Offset between SkyField and Astropy: 20.16232329399357 arcsec

Orbit: 2
Orbit start: 2017-07-18T14:40:20 Orbit end: 2017-07-18T15:42:10
Aim time: 2017-07-18T15:11:15 RA (deg): 117.98162045586982 deg Dec (deg): 20.940012769306016 deg
SkyField Aim time: 2017-07-18T15:11:15 RA (deg): 117.98751219671894 deg Dec (deg): 20.938969263324555 deg

Offset between SkyField and Astropy: 20.162

# Okay, now check to see what the parallax does in each orbit.

### Compare Astropy/Sunpy to what you get when you correct for the orbital parallax. Every step below is 100 seconds.



In [16]:
from astropy.coordinates import SkyCoord
from datetime import timedelta


for ind, orbit in enumerate(orbits):
    midTime = orbit[0]
    while(midTime < orbit[1]):
        
        sky_pos = planning.get_sky_position(midTime, offset)

        skyfield_pos = planning.get_skyfield_position(midTime, offset, load_path='../data', parallax_correction=True)
        skyfield_geo = planning.get_skyfield_position(midTime, offset, load_path='../data', parallax_correction=False)

        
        skyfield_ephem = SkyCoord(skyfield_pos[0], skyfield_pos[1])
        skyfield_geo_ephem = SkyCoord(skyfield_geo[0], skyfield_geo[1])
#        sunpy_ephem = SkyCoord(sky_pos[0], sky_pos[1])
        print('Offset between parallax-corrected positions and geoenctric is {} arcsec'.format(
            skyfield_geo_ephem.separation(skyfield_ephem).arcsec)
             )
        dra, ddec = skyfield_geo_ephem.spherical_offsets_to(skyfield_ephem)
        print('{0} delta-RA, {1} delta-Dec'.format(dra.to(u.arcsec), ddec.to(u.arcsec)))
        print('')
        midTime += timedelta(seconds=100)
        
    break

Offset between parallax-corrected positions and geoenctric is 8.664131980754892 arcsec
8.654283660118677 arcsec delta-RA, -0.4129858468939972 arcsec delta-Dec

Offset between parallax-corrected positions and geoenctric is 9.00997682125181 arcsec
9.009464612674924 arcsec delta-RA, -0.09607139237710091 arcsec delta-Dec

Offset between parallax-corrected positions and geoenctric is 9.261767536115142 arcsec
9.259108154863679 arcsec delta-RA, 0.22193258081934353 arcsec delta-Dec

Offset between parallax-corrected positions and geoenctric is 9.415587542643516 arcsec
9.400244500410418 arcsec delta-RA, 0.5373007587981358 arcsec delta-Dec

Offset between parallax-corrected positions and geoenctric is 9.469071719804337 arcsec
9.431173504717664 arcsec delta-RA, 0.8463365522920199 arcsec delta-Dec

Offset between parallax-corrected positions and geoenctric is 9.421373066533983 arcsec
9.35148630520005 arcsec delta-RA, 1.145414485410706 arcsec delta-Dec

Offset between parallax-corrected positions a